In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## fastapi 를 설치한다

In [ ]:
! pip install fastapi uvicorn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 7.7 MB/s eta 0:00:00


In [ ]:
import os
import glob
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from PIL import Image

## 이미지데이터 위치

In [ ]:
# 데이터 경로
data_dir = '/content/drive/MyDrive/crack/crack_1000'
model_file = '/content/drive/MyDrive/crack/resnet18_weights.pth'


# 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"device={device}")


device=cuda


## 딥러닝 모델 정의 및 weight 로딩

In [ ]:
# 1. 모델 정의 및 가중치 로드
model = models.resnet18(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, 2)  # 이진 분류를 위한 출력 레이어 수정
model.load_state_dict(torch.load(model_file, map_location=device))
model.eval()  # 추론 모드로 전환


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
<ipython-input-5-b955846dfe61>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions 

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:

# 2. 이미지 전처리 함수 정의
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

## 추론하기 (이미지 classification)
*   이미지 파일 읽기
*   추론하기 (predict)
*   2개의 predict 값 중에서 큰값으로 positive, negative 결정하기


In [ ]:
def inference(image_name):
    # 3. 이미지 로드 및 전처리
    image_path = image_name  # 추론할 이미지 파일 경로
    image = Image.open(image_path).convert('RGB')  # 이미지 로드 및 RGB로 변환
    input_tensor = preprocess(image)  # 전처리 수행
    input_batch = input_tensor.unsqueeze(0)  # 배치 차원 추가 (1, C, H, W)

    # 4. 모델을 통한 추론
    with torch.no_grad():  # 추론이므로 그라디언트 비활성화
        output = model(input_batch)
        print(output)
        # 두개 값에 중에 큰 값을 선택한다
        _, predicted_class = torch.max(output, 1)

    # 5. 결과 출력
    class_names = ['negative', 'positive']  # 클래스 이름 정의
    print(f"AI: {image_name} --> {class_names[predicted_class.item()]}")
    return class_names[predicted_class.item()]


In [ ]:
# 샘플로 확인해보기 1
inference(data_dir + '/negative/00020.jpg')

tensor([[ 4.2426, -4.7941]])
AI: /content/drive/MyDrive/crack/crack_1000/negative/00020.jpg --> negative


'negative'

In [ ]:
# 샘플로 확인해보기 2
inference(data_dir + '/positive/00020.jpg')

tensor([[-8.9168,  9.1864]])
AI: /content/drive/MyDrive/crack/crack_1000/positive/00020.jpg --> positive


'positive'

# fastapi 로 서빙하기

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
from typing import List

import requests
import uvicorn

app = FastAPI()

## colab에서 fastapi를 사용할 때 nest_asyncio 필요



In [ ]:
# chatgpt에서 알려줌
import nest_asyncio
nest_asyncio.apply()

## 이미지대문 페이지
*  인사말 하기

In [ ]:
# 대문페이지 (/ 루트 페이지)
@app.get("/")
async def get_index():
    return "hello. this image AI demo page !!"


## 이미지 인식 페이지
*    '/imageai/positive/00001.jpg' 이런식으로 입력해야 함

In [ ]:
# 이미지 인식 페이지
@app.get("/imageai/{label}/{img}", )
async def get_image_ai(label: str, img: str):
    # 이미지 파일 경로를 지정합니다.
    # ~~~~~/positive/00010.jpg
    # 데이터 경로
    global data_dir

    image_path = f"{data_dir}/{label}/{img}"

    # 이미지 파일이 존재하는지 확인합니다.
    if os.path.exists(image_path):
        # 파일이 존재하면 파일 응답을 반환합니다.
        print(f"file {image_path} ok")
        inf_result = inference(image_path)
        result = f"이미지 인식 결과: {img} --> {inf_result}"
        return {"result": result}

    else:
        # 파일이 존재하지 않으면 404 오류를 반환합니다.
        return {"error": f"Image not found - {image_path}"}

In [ ]:
# FastAPI 서버를 별도의 스레드에서 실행
import threading

def run():
    uvicorn.run(app, host="0.0.0.0", port=8000)

# FastAPI 서버 실행 스레드
threading.Thread(target=run).start()


## ngrok 설치
*   **colab에서** 띄운 fastapi 웹서버를 접근할 수 있도록 한다.



In [ ]:
! pip install pyngrok


INFO:     Started server process [552]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


- 고유 authtoken값을 입력해야한다.

In [ ]:
! ngrok authtoken "2o32G2lOJ4lXPa5m7UOncTE7oNi_5DJPaC9w4nhvzcFzq89ZZ"


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from pyngrok import ngrok

# ngrok을 통해 포트 8000에 접근
public_url = ngrok.connect(8000)
print("Public URL:", public_url)

## 성공하면 이렇게 나옴
## Public URL: NgrokTunnel: "https://db46-34-145-66-177.ngrok-free.app" -> "http://localhost:8000"


Public URL: NgrokTunnel: "https://6221-34-143-178-182.ngrok-free.app" -> "http://localhost:8000"
